In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-122065")
exp = Experiment(workspace=ws, name="quick-starts-ws-122065")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

# run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code R62SYSCQY to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-122065
Azure region: southcentralus
Subscription id: 2552278b-2817-43a7-820e-5a5a53ff9e19
Resource group: aml-quickstarts-122065


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

comptarg = ComputeTarget(workspace=ws, name='Ranga')
comp_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",max_nodes=4)
comp_cluster = ComputeTarget.create(ws,'cluster',comp_config)

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, randint, choice
from sklearn.metrics import mean_squared_error,accuracy_score
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
    'C': choice(0.01,0.1,1,10,100),
    'max_iter': choice(50,100,150,200)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.01)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(entry_script='train_LATEST_FINAL.py',source_directory='./training',compute_target=comp_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,primary_metric_name='Accuracy',primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=20,policy=policy,estimator=est)

In [23]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

exp = Experiment(workspace=ws,name='Ranga_Hyperdrive')
run = exp.submit(hyperdrive_config,show_output=True)
RunDetails(run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [26]:
print(run.get_portal_url())

https://ml.azure.com/experiments/Ranga_Hyperdrive/runs/HD_6b14aa83-22c3-45ef-9571-9f06dbe11ba0?wsid=/subscriptions/8e713106-916f-4177-890e-435b90d7adc4/resourcegroups/aml-quickstarts-121863/workspaces/quick-starts-ws-121863


In [27]:
import joblib
# Get your best run and save the model from that run.

best = run.get_best_run_by_primary_metric()

In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

webpaths = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'

datadss = TabularDatasetFactory.from_delimited_files(path=webpaths,validate='False',separator=',',infer_column_types=True,include_path=False,
set_column_types=None,support_multi_line=False,partition_format=None)

ds = datadss.to_pandas_dataframe()

In [6]:
from train_LATEST_FINAL import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
cleands = x.join(y)

cleandstrain,cleandstest = train_test_split(cleands,test_size=0.33,random_state=42)

In [7]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=25,
    task='classification',
    primary_metric='accuracy',
    training_data=cleandstrain,
    label_column_name='y',
    n_cross_validations=3)

In [8]:
# Submit your automl run

autoexp = Experiment(workspace=ws,name='Ranga_AutoML')
autorun = autoexp.submit(automl_config)
RunDetails(autorun).show()

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [9]:
print(autorun.get_portal_url())

https://ml.azure.com/experiments/Ranga_AutoML/runs/AutoML_91157af8-7684-4b1d-bbab-2d476711feff?wsid=/subscriptions/2552278b-2817-43a7-820e-5a5a53ff9e19/resourcegroups/aml-quickstarts-122065/workspaces/quick-starts-ws-122065


In [10]:
# Retrieve and save your best automl model.

best,fitted = autorun.get_output()
print(best)

Run(Experiment: Ranga_AutoML,
Id: AutoML_91157af8-7684-4b1d-bbab-2d476711feff_38,
Type: None,
Status: Completed)


In [11]:
print(fitted)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  n_jobs=1,
                                                                                                  penalty='none',
                                                                                                  power_t=0,
                                                             

In [ ]:
comp_cluster.delete()